## SVC (support vector classifier) with GridSearchCV

In [1]:
# read the dataset
import pandas as pd 
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [2]:
# convert categorical into numerical 
dataset = pd.get_dummies(dataset, drop_first=True, dtype=int)
dataset = dataset.drop(["User ID"],axis=1)
dataset.head()

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1


In [3]:
# split input and output 
independent = dataset[[ "Age",	"EstimatedSalary", "Gender_Male"]]
dependent = dataset[[ "Purchased" ]]

In [4]:
# Check dataset is balanced or Imbalanced
dataset["Purchased"].value_counts()    # imbalanced 

Purchased
0    257
1    143
Name: count, dtype: int64

In [5]:
# split train and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(independent,dependent, test_size=0.20,random_state=0)
print(x_train,x_test,y_train,y_test )

     Age  EstimatedSalary  Gender_Male
336   58           144000            1
64    59            83000            0
55    24            55000            0
106   26            35000            0
300   58            38000            0
..   ...              ...          ...
323   48            30000            0
192   29            43000            1
117   36            52000            1
47    27            54000            0
172   26           118000            0

[320 rows x 3 columns]      Age  EstimatedSalary  Gender_Male
132   30            87000            1
309   38            50000            0
341   35            75000            1
196   30            79000            0
246   35            50000            0
..   ...              ...          ...
14    18            82000            1
363   42            79000            0
304   40            60000            0
361   53            34000            0
329   47           107000            0

[80 rows x 3 columns]      Purchased
33

## standardization

In [6]:
# standardization 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## save the standardization for deployment purpose

In [7]:
import pickle
pickle.dump(sc, open("SVC_purchase_scaler.sav","wb"))

In [8]:
y_train = y_train.values.ravel()

In [9]:
# model creation + grid cv
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC 
param_grid = {
"kernel": ["linear", "poly", "rbf", "sigmoid"],  #precomputed - need x to be a square matrix 
"gamma": ["scale", "auto"]
}
model = SVC(probability=True)

grid_model_SVC= GridSearchCV(model, param_grid,cv=5, refit=True , verbose=3, n_jobs =-1, scoring='f1_weighted') 
grid_model_SVC.fit(x_train,y_train)


Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [10]:
# model prediction + grid cv
grid_prediction = grid_model_SVC.predict(x_test)
grid_prediction

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1])

In [11]:
# model evaluation --- Performance on the unseen TEST dataset
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test,grid_prediction)
clf_report = classification_report(y_test, grid_prediction)
print(cm)
print(clf_report)

[[55  3]
 [ 1 21]]
              precision    recall  f1-score   support

           0       0.98      0.95      0.96        58
           1       0.88      0.95      0.91        22

    accuracy                           0.95        80
   macro avg       0.93      0.95      0.94        80
weighted avg       0.95      0.95      0.95        80



## ROC AUC 

In [12]:
print(grid_model_SVC.predict_proba(x_test)[:,1])

[0.06934943 0.03834154 0.02509619 0.03077069 0.01629205 0.04714131
 0.07277763 0.96323786 0.04583535 0.65697823 0.00533848 0.0093119
 0.01325572 0.07895901 0.07193151 0.73889648 0.04895504 0.07334002
 0.88485066 0.10762755 0.01811297 0.88850744 0.10620511 0.94165378
 0.03653572 0.88719939 0.06583503 0.02667652 0.02428136 0.0665119
 0.07881885 0.06024858 0.93551973 0.01375368 0.00897609 0.03204547
 0.04137549 0.01751241 0.01906593 0.88962785 0.06944749 0.06065063
 0.05100519 0.13205938 0.79657112 0.01034179 0.06324607 0.81804722
 0.02046368 0.84382963 0.88375611 0.06079648 0.01465133 0.88890065
 0.86850937 0.68184641 0.04556105 0.03635672 0.93478422 0.05551634
 0.08577768 0.95673669 0.04351692 0.54590456 0.07004771 0.93874284
 0.081707   0.00588229 0.05386828 0.12727909 0.91340342 0.03657996
 0.02174457 0.85204359 0.07884459 0.0679038  0.17985382 0.06164561
 0.95067927 0.81106654]


In [13]:
from sklearn.metrics import roc_auc_score     # key metric for binary classification 
roc_auc_score = roc_auc_score(y_test,grid_model_SVC.predict_proba(x_test)[:,1] )
roc_auc_score

np.float64(0.9678683385579937)

## f1 score 

In [14]:
from sklearn.metrics import f1_score
f1_score = f1_score(y_test,grid_prediction, average='weighted' )
f1_score

0.950648360030511

## GridSearchCV results 

In [15]:
grid_results = grid_model_SVC.cv_results_
print(grid_results)

{'mean_fit_time': array([0.08117723, 0.02386518, 0.02552919, 0.03909483, 0.02033086,
       0.02141671, 0.02453804, 0.02489023]), 'std_fit_time': array([0.07370182, 0.00165532, 0.00173059, 0.0193571 , 0.00166427,
       0.00159295, 0.00236005, 0.00322859]), 'mean_score_time': array([0.01623073, 0.00991173, 0.01116924, 0.0120533 , 0.00992346,
       0.00999541, 0.01059299, 0.00926142]), 'std_score_time': array([0.00719841, 0.0009482 , 0.00201834, 0.00293956, 0.00129101,
       0.00207944, 0.00040421, 0.00147377]), 'param_gamma': masked_array(data=['scale', 'scale', 'scale', 'scale', 'auto', 'auto',
                   'auto', 'auto'],
             mask=[False, False, False, False, False, False, False, False],
       fill_value=np.str_('?'),
            dtype=object), 'param_kernel': masked_array(data=['linear', 'poly', 'rbf', 'sigmoid', 'linear', 'poly',
                   'rbf', 'sigmoid'],
             mask=[False, False, False, False, False, False, False, False],
       fill_value=np.

In [16]:
# table
table = pd.DataFrame.from_dict(grid_results)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.081177,0.073702,0.016231,0.007198,scale,linear,"{'gamma': 'scale', 'kernel': 'linear'}",0.796088,0.758892,0.740864,0.820367,0.902824,0.803807,0.056776,5
1,0.023865,0.001655,0.009912,0.000948,scale,poly,"{'gamma': 'scale', 'kernel': 'poly'}",0.794154,0.783708,0.769318,0.873807,0.919631,0.828124,0.058387,3
2,0.025529,0.001731,0.011169,0.002018,scale,rbf,"{'gamma': 'scale', 'kernel': 'rbf'}",0.860542,0.890137,0.859920,0.907389,0.968750,0.897348,0.040016,1
3,0.039095,0.019357,0.012053,0.002940,scale,sigmoid,"{'gamma': 'scale', 'kernel': 'sigmoid'}",0.766401,0.795968,0.720769,0.726744,0.842259,0.770428,0.045172,7
4,0.020331,0.001664,0.009923,0.001291,auto,linear,"{'gamma': 'auto', 'kernel': 'linear'}",0.796088,0.758892,0.740864,0.820367,0.902824,0.803807,0.056776,5
5,0.021417,0.001593,0.009995,0.002079,auto,poly,"{'gamma': 'auto', 'kernel': 'poly'}",0.794154,0.783708,0.769318,0.873807,0.919631,0.828124,0.058387,3
6,0.024538,0.002360,0.010593,0.000404,auto,rbf,"{'gamma': 'auto', 'kernel': 'rbf'}",0.860542,0.890137,0.859920,0.907389,0.968750,0.897348,0.040016,1
7,0.024890,0.003229,0.009261,0.001474,auto,sigmoid,"{'gamma': 'auto', 'kernel': 'sigmoid'}",0.766401,0.795968,0.720769,0.726744,0.842259,0.770428,0.045172,7


In [17]:
grid_model_SVC.best_params_

{'gamma': 'scale', 'kernel': 'rbf'}

In [18]:
grid_model_SVC.best_estimator_

SVC(probability=True)

In [19]:
""" Mean cross-validation score on the TRAINING data """
grid_model_SVC.best_score_

np.float64(0.8973475249246045)

## model prediction with real time data

In [ ]:
## user input 
Gender=input("Enter your gender (0-female/1-male): ")
Age	= int(input("Enter your age: "))
EstimatedSalary = int(input("Enter your estimated salary: "))


""" test input data's
Male	19	19000	- 0
Female	46	41000	- 1
Female	50	20000	- 1
Male	36	33000	- 0
"""

In [21]:
## user input predction 
user_input_predction = grid_model_SVC.predict([[Gender,Age,EstimatedSalary ]])
if user_input_predction==1:
    print("user will purchase")
else:
    print("user will not purchase")

print(user_input_predction)

user will purchase
[1]


# Saving the Final Model 

In [22]:
import pickle
pickle.dump(grid_model_SVC, open("grid_model_SVC_final.sav", "wb"))